In [1]:
1

1

In [2]:
prog = "1+1"
ex1 = Meta.parse(prog)

:(1 + 1)

In [3]:
ex1.head, ex1.args

(:call, Any[:+, 1, 1])

In [4]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}(

(3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [6]:
ex3 = Meta.parse("(4 + 4) / 2")
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

In [7]:
a = 1 
:($a + b)

:(1 + b)

In [8]:
ex = :(1 + 2)
eval(ex)

3

In [10]:
function make_expr2(op, op1, op2)
    op1f, op2f = map(x -> isa(x, Number) ? 2*x : x, (op1, op2))
    Expr(:call, op, op1f, op2f)
end 

make_expr2(:+, 1, Expr(:call, :*, 5, 8))

:(2 + 5 * 8)

In [12]:
macro sayhello()
    :( println("hello") )
end 

@sayhello()

hello


In [14]:
macro sayhello(name)
    :( println("hello ", $name) )
end 
@sayhello("human")

hello human


In [15]:
macro twostep(arg)
    println("parse time")
    :( println("runtime") )
end 

ex = macroexpand(Main, :(@twostep :(1, 2, 3)));

parse time


In [16]:
eval(ex)

runtime


In [17]:
macro assert(ex)
    :( $ex ? nothing : throw(AssertionError($(string(ex)))) )
end 

@assert 1 == 1. 
@assert 1 == 0

AssertionError: AssertionError: 1 == 0

In [20]:
macro time(ex)
    return quote 
        local t0 = time_ns()
        local val = $(esc(ex))
        local t1 = time_ns()
        println("time: ", (t1 - t0)/1e9, " seconds")
        val 
    end 
end 


f() = sum(1:100)
@time f()


time: 0.006970793 seconds


5050

In [21]:
macro zerox()
    return esc(:(x = 0))
end 

function foo()
    x = 1 
    @zerox 
    x 
end 
foo()

0

In [26]:
function timeit(f)
    t0 = time_ns()
    val = f()
    t1 = time_ns()
    println("time: ", (t1 - t0)/1e9, " seconds")
    val 
end 

macro time(exp)
    :(timeit(() -> $(esc(exp))))
end 

@time sum(1:1000)

time: 7.0e-8 seconds


500500

In [27]:
macro m(::Int)
    println("int")
end 

macro m(args...)
    println("$(length(args))")
end 

@m 1 

x = 2 
@m x 

int
1


In [30]:
struct MyNumber 
    x::Float64 
end 

for op in (:sin, :cos, :tan, :log, :exp)
    eval(quote
        Base.$op(a::MyNumber) = MyNumber($op(a.x))
    end)
end 

x = MyNumber(0.)
cos(x), sin(x)

(MyNumber(1.0), MyNumber(0.0))

In [31]:
@generated function foo(x)
    Core.println(x)
    :(x * x)
end 

x = foo(2);

Int64


In [32]:
x

4

In [33]:
y = foo("bar")

String


"barbar"

In [34]:
foo(4)

16

In [36]:
f(x) = "original"

g(x) = f(x) 
@generated g1(x) = f(x)
@generated g2(x) = :(f(x))

f(x::Int) = "int"

f(1), g1(1), g2(1)

("int", "original", "int")

In [37]:
function sub2ind_loop(dims::NTuple{N}, I::Integer...) where N 
    ind = I[N] - 1 
    for i in N-1:-1:1
        ind = I[i] - 1 + dims[i] * ind 
    end 
    ind + 1
end 
sub2ind_loop((3, 5), 1, 2)

4

In [43]:
sub2ind_rec(dims::Tuple{}) = 1
sub2ind_rec(dims::Tuple{}, i1::Integer, I::Integer...) = 
    i1 == 1 ? sub2ind_rec(dims, I...) : throw(BoundsError())
sub2ind_rec(dims::Tuple{Integer, Vararg{Integer}}, i1::Integer) = i1 
sub2ind_rec(dims::Tuple{Integer, Vararg{Integer}}, i1::Integer, I::Integer...) = 
    i1 + dims[1] * (sub2ind_rec(Base.tail(dims), I...) - 1)

sub2ind_rec((3, 5), 1, 2)


4

In [44]:
@generated function sub2ind_gen(dims::NTuple{N}, I::Integer...) where N 
    ex = :(I[$N] - 1)
    for i in N-1:-1:1
        ex = :(I[$i] - 1 + dims[$i] * $ex)
    end 
    :($ex + 1)
end 

sub2ind_gen((3, 5), 1, 2)

4